In [17]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [26]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])

traindsets = datasets.MNIST('../data/', download = True, train = True, transform= transform)
testdsets= datasets.MNIST('../data/', download = True, train = False, transform= transform)

In [27]:
trainLoader = torch.utils.data.DataLoader(traindsets, batch_size=4,
                                         shuffle= True, num_workers= 2)
testLoader = torch.utils.data.DataLoader(testdsets, batch_size=4,
                                        shuffle = False, num_workers=2)

In [23]:
model= Net()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs= 1

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [24]:
for epoch in range(num_epochs):
    running_loss=0.0
    for i, (img, target) in enumerate(trainLoader):
        img = Variable(img)
        target= Variable(target)
        model.zero_grad()
        outputs = model(img)
        loss = criterion(outputs, target)
        loss.backward()
        running_loss += loss.data[0]
        optimizer.step()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # Remove the CWD from sys.path while we load stuff.


[1,  2000] loss: 0.538
[1,  4000] loss: 0.418
[1,  6000] loss: 0.331
[1,  8000] loss: 0.307
[1, 10000] loss: 0.288
[1, 12000] loss: 0.883
[1, 14000] loss: 0.777


In [28]:
total =0
correct = 0
for data in testLoader:
    img, labels = data
    img= Variable(img)
    labels = Variable(labels)
    outputs = model(img)
    _, predicted = torch.max(outputs.data , 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 85 %
